# Playground #3 - Monitoring, evaluation, app deployment.

In [ ]:
# Dependencies installation
!pip install langchain langchain-community langchain-openai tiktoken langfuse lancedb langgraph docling langchain-docling gradio python-dotenv

In [ ]:
import os
from google.colab import userdata
# Set environment variables from secrets
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_HOST"] = userdata.get("LANGFUSE_HOST")

In [ ]:
!gsutil cp -r gs://gid-rag-platform-llmops-workshops/dummy.json sa_key.json

## 1. App Setup

In [ ]:
# access existing knowledge bases

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import LanceDB
import lancedb


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "sa_key.json"

KNOWLEDGE_BASES= {
  "WIZARD_OF_OZ": "gs://gid-rag-platform-llmops-workshops/wizard_of_oz",
  "ODDYSEY": "gs://gid-rag-platform-llmops-workshops/oddysey",
  "PRAWO_O_RUCHU_DROGOWYM": "gs://gid-rag-platform-llmops-workshops/prawo_o_ruchu_drogowym"
}

def get_vector_store_by_name(name: str):
  connection = lancedb.connect(KNOWLEDGE_BASES['ODDYSEY'])
  return LanceDB(
      connection=connection,
      embedding=OpenAIEmbeddings(),
      table_name="knowledge_base"
  )



In [ ]:
# play around with vector_store
get_vector_store_by_name('ODDYSEY').similarity_search("Who was Calypso?", k=5)


In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI


# Define graph state
class RAGState(TypedDict):
  knowledge_base: str
  no_snippets: int
  question: str
  snippets: list
  answer: str


# Node #1 - retrieval
def retrieve_snippets(state: RAGState):
  no_snippets = state.get("no_snippets", 3)
  retriever = get_vector_store_by_name(state["knowledge_base"]).as_retriever(search_kwargs={"k": no_snippets})
  snippets = retriever.get_relevant_documents(state["question"])
  return {"snippets": snippets}


# Node #2 - generation
def generate_answer(state: RAGState):
  context = "\n\n".join(f"text: {doc.page_content}\n page: {doc.metadata['page_no']}" for doc in state["snippets"])
  prompt = f"Using the context below, answer the question and provide the information which page was used for creating the answer.\n\nQuestion: {state['question']}\nContext: {context}\nAnswer:"
  answer = ChatOpenAI(temperature=0.5, model_name="gpt-4o-mini").invoke(prompt).content
  return {"answer": answer}


# Build the state graph with sequential nodes
graph_builder = StateGraph(RAGState).add_sequence([retrieve_snippets, generate_answer])
graph_builder.add_edge(START, "retrieve_snippets")
graph_builder.add_edge("generate_answer", END)
graph = graph_builder.compile()


graph.invoke({"question": "Who was Calypso?", "knowledge_base": "ODDYSEY"})

# 2. Monitoring LLM application

In [ ]:
# Instrumenting the application with monitoring
from langfuse.callback import CallbackHandler
import gradio as gr

MY_USER_ID = "mateusz"

langfuse_handler = CallbackHandler(trace_name="Workshops", user_id=MY_USER_ID)

def answer_oz_question(question: str, no_snippets: int, knowledge_base: str):
    response = graph.invoke(
        {'question': question, 'no_snippets': no_snippets, 'knowledge_base': knowledge_base},
        config={"callbacks": [langfuse_handler]})
    return response['answer']

demo = gr.Interface(
    fn=answer_oz_question,
    inputs=[
        gr.Textbox(label="Question"),
        gr.Slider(label="Number of snippets", minimum=1, maximum=10, step=1, value=3),
        gr.Dropdown(label="Knowledge Base", choices=list(KNOWLEDGE_BASES.keys()), value="WIZARD_OF_OZ")
    ],
    outputs=gr.TextArea(label="Answer", interactive=False)
)
demo.launch(share=True)

Let's now go to [https://cloud.langfuse.com](https://cloud.langfuse.com) and explore the traces.

# 3. Evaluating prompts

Demonstration of LLM-as-judge with Langfuse

# 4. Prompt Management


Let's create a new prompt in langfuse prompt management [link](https://cloud.langfuse.com/project/cm8ymgiyi01osad069zauomkl/prompts). Use your name as a prefix so we can differentiate.

In [ ]:
from langfuse import Langfuse
langfuse = Langfuse()

prompt = langfuse.get_prompt("mateuszpytel-prompt-test")
prompt.compile(question="<question_here>", context="<context_here>")

* EXERCISE: Try modifying the application code above to use prompt from langfuse. Play around with different prompts, e.g. ask to answer in different language or in a funny way, like with rhymes for example. Observe how it impacts the score.


# 5. Application deployment
Demonstration on how to develop the app and deploy it to huggingfaces.

1. Create a huggingface account
2. Checkout the repo, go to `exercises/03-deployment`
3. Run `gradio deploy` and follow the instructions